In [ ]:
using CABLAB
include("Optimize_NPP.jl")
registerDATFunction(optimize_npp,(TimeAxis,VariableAxis));
registerDATFunction(predict_npp,((TimeAxis,VariableAxis),()),(TimeAxis,)); 

Here, we want to show how the cube and the DAT might be used for model parameter estimation. It also shows how the mapCube function can be applied on multiple input data cubes. 

In the file *Optimize_NPP.jl* we defined a function called **predict_npp** which predicts the ecosystem's net productivity (**NPP**) based on air temperature, fpar and radiation using a very simple model. One of the model parameters is the optimal growing temperature **Topt** at which the productivity reaches its maximum. We assume that this parameter varies across ecosystems, because they adapt to their normal environmental conditions. 

In [ ]:
c             = Cube("/Net/Groups/BGI/scratch/fgans/cubecopy/datacube/")
vars          = ["t2m","fpar","Rg"];
cdata         = getCubeData(c,variable=vars,longitude=(-10,35),latitude=(35,65));

In [ ]:
cdata_filled  = mapCube(gapFillMSC,cdata);

In the first experiment we assume that the model parameter **Topt** equals the mean temperature of each grid cell.

In [ ]:
tair=getCubeData(c,variable="t2m",longitude=(-10,35),latitude=(35,65))
mtair=reduceCube(mean, tair, TimeAxis, skipnull=true);

We pass the mean air temperature mtair as a second cube argument to the **mapCube** function.

In [ ]:
predict_nppmean=mapCube(predict_npp,(cdata_filled,mtair));

Now, we can plot time series and maps of the predicted NEE. 

In [ ]:
plotTS(predict_nppmean)

In [ ]:
using Compose
plot = plotMAP(predict_nppmean)
draw(SVG("plot.svg", 6cm, 4cm), plot)

Now we want to estimate the optimal growing temperature differently. Instead of just calculating the mean temperature we choose **Topt** so that the time mean of **NEE** is maximised for each gride cell. We do this by wrapping an optimization method from Julia's Optim package. 

In [ ]:
topt=mapCube(optimize_npp,cdata_filled);

And we get a map of the optimal growing temperature

In [ ]:
plotMAP(topt)

And the resulting NEE

In [ ]:
predict_nppopt=mapCube(predict_npp,(cdata_filled,topt));

In [ ]:
plotMAP(predict_nppopt)

In order to calculate the global map of Topt we add a few processors and repate the saem steps on the global cube

In [ ]:
#using ClusterBGC
#addprocs_lsf(20,queue="BTM")
addprocs(10)
@everywhere include("Optimize_NPP.jl")

In [ ]:
CABLAB.DAT.init_DATworkers()

In [ ]:
cdata         = getCubeData(c,variable=vars);
@time cdata_filled  = mapCube(gapFillMSC,cdata);
@time topt=mapCube(optimize_npp,cdata_filled,max_cache=1e7);

We can compare the optimum growing temperature (first plot) with the mean air temperature (second plot). In very cold regions you see that **Topt** is higher, probably because of the low temperatures occur when fpar and radiation are also very low and carbon uptake is not possible. 

In [ ]:
plotMAP(topt,dmin=0.f0,dmax=30.f0)

In [ ]:
tair=getCubeData(c,variable="t2m")
@time mtair=reduceCube(mean, tair, TimeAxis, skipnull=true);
plotMAP(mtair,dmin=0.f0,dmax=30.f0)

In [ ]:
rmprocs(workers())